In [ ]:
# default_exp core.scraping.linescore

# Linescore

> Scrapes CZ linescore information.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

from czapi.core.scraping.base import make_soup
from czapi.core.scraping.constants import BOXSCORE_KWARGS
from bs4 import BeautifulSoup, Tag
from collections import defaultdict
from typing import List, Union

In [ ]:
#export
def generate_dict_from_table(

    table : Tag

)->Union[dict,defaultdict]:
    """Helper function for returning the curling boxscore from a bs4 Tag object."""
    d = defaultdict(list)
    team = None
    
    # TODO : add error handling for when no table is passed / None
    
    if table is None:
        raise ValueError('Table tag is NoneType.')
    
    # loop through tags in table
    for tag in table.find_all('td'):
        if tag.attrs.get('class') == ['linescoreteam']:
            team = tag.a.string
            d[team] = defaultdict(list)
            d[team]['href'] = tag.a['href']
        elif tag.attrs.get('class') == ['linescorehammer']:
            d[team]['hammer'] = not bool(tag.string) # opposite for some reason
        elif tag.attrs.get('class') == ['linescoreend']:
            d[team]['score'].append(tag.string.strip())
        elif tag.attrs.get('class') == ['linescorefinal']:
            d[team]['finalscore'] = tag.b.string.strip()
        
    return d

In [ ]:
#export

def get_boxscore_from_table(

    table : Tag

)->Union[dict,defaultdict]:
    """Wraps generate_dict_from_table for clarity / error handling."""
    try:
        return generate_dict_from_table(table = table)
    
    except ValueError as e:
        # TODO : change return value based on what makes sense for the API
        return {}

def get_boxscore_from_game_id(

     cz_game_id : str
    ,**request_kwargs
)->Union[dict,defaultdict]:
    """Returns a curling boxscore (dict) based on the cz_game_id."""
    
    url = 'https://www.curlingzone.com/game.php?1=1&showgameid=%s#1'%cz_game_id
    soup = make_soup(url=url,**request_kwargs)
    table = soup.find(**BOXSCORE_KWARGS)
    
    
    return get_boxscore_from_table(table=table)


> The picture below highlights where the cz_game_id is found in the CurlingZone [URL](https://curlingzone.com/game.php?1=1&showgameid=271145#1). 

![game by gameid](./imgs/game_by_game_id.png)

In [ ]:
expected_dict = {
    
    'Wayne Tuck Jr.' : {
        
         'href' : 'event.php?view=Team&eventid=6400&teamid=144353&profileid=12486#1'
        ,'hammer' : True
        ,'score' : ['0','2','0','0','0','0','1','1','1','0']
        ,'finalscore' : '5'
        
    }
    ,'Matthew Hall' : {
        
         'href' : 'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435#1'
        ,'hammer' : False
        ,'score' : ['0','0','4','0','0','1','0','0','0','2']
        ,'finalscore' : '7'
    }
    
}

actual_dict = get_boxscore_from_game_id(cz_game_id = 271145)

assert actual_dict == expected_dict

In [ ]:
#export

def get_table_from_index(

     tables : List[Tag]
    ,game_number : int

)->Tag:
    """Returns a 'table' Tag object from a list of 'table' Tag objects. This helper function allows for 1 indexing instead of 0."""
    # TODO confirm this is the kind of error handling we want
    if game_number < 1 :
        raise ValueError('Table number must be greater than 0.')
        
    game_idx = game_number -1 
    
    try:
        return tables[game_idx]
    except IndexError as e:
        raise IndexError(". ".join([str(e),"Are you sure that game number is valid?"]))
    
def get_boxscore_from_event_draw_game_number(

     cz_event_id : str
    ,cz_draw_id : int
    ,game_number : int 
    ,**request_kwargs
):
    """Returns a curling boxscore (dict) based on the cz_event_id, cz_draw_id and game_number."""
    url = 'https://curlingzone.com/event.php?eventid=%s&view=Scores&showdrawid=%s#1'%(cz_event_id,cz_draw_id)
    soup = make_soup(url=url,**request_kwargs)
    
    tables = soup.find_all(**BOXSCORE_KWARGS)
    try:
        table = get_table_from_index(tables = tables, game_number = game_number)
        return get_boxscore_from_table(table = table)
    
    except IndexError as e:
        return {}
    
    except ValueError as e:
        return {}  

> The picture below highlights where the cz_event_id and cz_draw_id are found and how the games are numbered (game_number) on the CurlingZone [page](https://curlingzone.com/event.php?eventid=6400&view=Scores&showdrawid=2#1). 

![game by event id, draw and game number](./imgs/game_by_event_draw_game_number.png)

In [ ]:
actual_dict =  get_boxscore_from_event_draw_game_number(

     cz_event_id = 6400
    ,cz_draw_id = 2
    ,game_number = 1

) == expected_dict